In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

def sabr_implied_vol(F, K, T, alpha, beta, rho, nu, eps=1e-12):
    if abs(F - K) < eps:
        # ATM approximate formula
        return alpha / (F**((1 - beta)/2))
    
    FK_avg = (F * K)**((1 - beta) / 2)
    log_FK = np.log(F / K)
    
    z = (nu / alpha) * FK_avg * log_FK
    x_z = np.log((np.sqrt(1 - 2*rho*z + z**2) + z - rho) / (1 - rho))
    
    if abs(z) < eps:
        zx = 1.0
    else:
        zx = z / x_z
    
    return (alpha / FK_avg) * zx

def sabr_calibration_error(params, F, strikes, T, market_vols):
    alpha, beta, rho, nu = params
    model_vols = [
        sabr_implied_vol(F, K, T, alpha, beta, rho, nu) 
        for K in strikes
    ]
    errors = np.array(market_vols) - np.array(model_vols)
    return errors

# Synthetic example
alpha_true = 0.2
beta_true  = 0.9
rho_true   = -0.2
nu_true    = 0.4

F = 100.0
T = 1.0
strikes = np.array([80, 90, 95, 100, 105, 110, 120])

# Generate synthetic data
synthetic_vols = [
    sabr_implied_vol(F, K, T, alpha_true, beta_true, rho_true, nu_true)
    for K in strikes
]
noise_level = 0.001
market_vols = np.array(synthetic_vols) + np.random.normal(0, noise_level, len(synthetic_vols))

initial_guess = [0.1, 0.5, 0.0, 0.3]
result = least_squares(
    sabr_calibration_error,
    x0=initial_guess,
    args=(F, strikes, T, market_vols),
    bounds=([1e-6, 0.0, -0.9999, 1e-6], [1.0, 1.0, 0.9999, 5.0])
)

alpha_calib, beta_calib, rho_calib, nu_calib = result.x

print("Calibration Results:")
print(f"alpha (σ0): {alpha_calib:.4f}")
print(f"beta       : {beta_calib:.4f}")
print(f"rho        : {rho_calib:.4f}")
print(f"nu (vol-vol): {nu_calib:.4f}")
print("\nOptimizer diagnostic:")
print(result.message)


Calibration Results:
alpha (σ0): 0.2033
beta       : 0.8959
rho        : -0.1844
nu (vol-vol): 0.4334

Optimizer diagnostic:
`gtol` termination condition is satisfied.
